In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CodeAlong').getOrCreate()

In [2]:
data =  spark.read.csv('College.csv',header=True,inferSchema=True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
for item in data.head(1)[0]:
    print(item)

Abilene Christian University
Yes
1660
1232
721
23
52
2885
537
7440
3300
450
2200
70
78
18.1
12
7041
60


In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(inputCols=['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad', 'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'],outputCol='features')

In [10]:
assem_data = assembler.transform(data)

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexed = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [13]:
output_data = indexed.fit(assem_data).transform(assem_data)#can also used pipeline. if code is repeated too much

In [14]:
output_data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = true)



In [16]:
final_data = output_data.select('features','PrivateIndex')

In [17]:
final_data.show()

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [18]:
train_data , test_data = final_data.randomSplit([.7,.3])

In [19]:
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier,GBTClassifier

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

In [29]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [21]:
from pyspark.ml import Pipeline

In [23]:
rf_clas = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features',numTrees=100)
dt_clas = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt_clas = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [42]:
acc_evalBC = BinaryClassificationEvaluator(labelCol='PrivateIndex')

## Random Forest

In [24]:
rf_model = rf_clas.fit(train_data)

In [25]:
rf_model.featureImportances

SparseVector(17, {0: 0.0527, 1: 0.0675, 2: 0.1426, 3: 0.0141, 4: 0.0098, 5: 0.1848, 6: 0.0686, 7: 0.2143, 8: 0.0529, 9: 0.006, 10: 0.0105, 11: 0.0136, 12: 0.015, 13: 0.0688, 14: 0.0255, 15: 0.0277, 16: 0.0256})

In [27]:
rf_pred = rf_model.transform(test_data)

In [30]:
acc_eval.evaluate(rf_pred)

0.9420289855072463

In [43]:
acc_evalBC.evaluate(rf_pred)

0.9832647462277092

## Decision Tree

In [31]:
dt_model = dt_clas.fit(train_data)

In [32]:
dt_pred = dt_model.transform(test_data)

In [33]:
acc_eval.evaluate(dt_pred)

0.9130434782608695

In [44]:
acc_evalBC.evaluate(dt_pred)

0.903360768175583

## GBT Classifer

In [34]:
gbt_model = gbt_clas.fit(train_data)

In [35]:
gbt_pred = gbt_model.transform(test_data)

In [36]:
acc_eval.evaluate(gbt_pred)

0.9227053140096618

In [40]:
acc_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',rawPredictionCol='prediction')

In [41]:
acc_eval2.evaluate(gbt_pred)

0.8783950617283951